In [3]:
TEXT_MAP = {
    'a': '11',
    'b': '12',
    'c': '13',
    'd': '14',
    'e': '15',
    'f': '16',
    'g': '17',
    'h': '18',
    'i': '19',
    'j': '20',
    'k': '21',
    'l': '22',
    'm': '23',
    'n': '24',
    'o': '25',
    'p': '26',
    'q': '27',
    'r': '28',
    's': '29',
    't': '30',
    'u': '31',
    'v': '32',
    'w': '33',
    'x': '34',
    'y': '35',
    'z': '36',
}

In [4]:
def reverse_dict(dict_org):
    dict_rev = {}
    for key in dict_org:
        val = dict_org[key]
        dict_rev[val] = key
    return dict_rev

In [5]:
NUMBER_MAP = reverse_dict(TEXT_MAP)

# predefine
BLOCK_SIZE = 5
LETTER_SIZE = 2
PRIME_SIZE = 6

In [6]:
def encode(text):
    numbers = []
    i = 0
    ns = ''
    blocks = [text[i:i+BLOCK_SIZE] for i in range(0,len(text),BLOCK_SIZE)]
    for b in blocks:
        ns = ''
        for t in b:
            n = TEXT_MAP[t]
            ns = ns+n
            i = i+1
        numbers.append(ns)
    return numbers

def decode(numbers):
    texts = ''
    for ns in numbers:
        for i in range(0,len(ns),LETTER_SIZE):
            sn = ns[i:i+LETTER_SIZE]
            t = NUMBER_MAP[sn]
            texts = texts+t
    return texts

In [7]:
import random
import math

def random_odd_number(n_digit):
    start = 10^(n_digit-1)/2
    end = 10^n_digit/2
    return 2 * random.choice(range(start,end)) + 1

# miller-rabin's probabilistic primality test - millers test
# strong pseudoprime
def rabins_primality_test(n,k):
    if n <= 1 or n == 4:
        return False

    if n == 3 or n == 5:
        return True
    
    t = n-1
    while t % 2 == 0:
        t = t/2
    
    def miller_test(b,t,n):
        x = power_mod(b,t,n)
        if x == 1 or x == n-1:
            return True
        return False

    passed = 0
    used_b = []
    for i in range(k):
        b = random.choice(range(5,n-1))
        while b in used_b:
            b = random.choice(range(5,n-1))
        if miller_test(b,int(t),n):
            passed += 1
    
    if passed/k < (1/4)^k:
        return False
    return True

def find_pq(prime_size):
    p = 1       #dummy value
    while not rabins_primality_test(p,10):
        p = random_odd_number(prime_size)
    q=1
    while not rabins_primality_test(q,10) or p == q: 
        q = random_odd_number(prime_size)
    return p,q 

def find_ek(p,q):
    # e bigger than p and q => (e,(p-1)(q-1))=1
    current_p = max(p,q)
    while True:
        ek = next_prime(current_p)
        if ek > math.log(p*q,2):
            return ek
    return 0  # dummy value

In [8]:
import timeit

# unit tests
## random_odd_number(n_digit) tests
print(len(str(random_odd_number(3))) == 3)
print(random_odd_number(3)%2 == 1)

## rabins_test(n,k) test
n = 201333667445
print(timeit.timeit(lambda: print(rabins_primality_test(n,10) == is_prime(n)), number=1))     # how is_prime() works in sage?


True
True
True
0.0002983340000000112


In [9]:
def encrypt(pnumbers):
    print(pnumbers)
    p,q = find_pq(PRIME_SIZE)
    n = p*q
    ek = find_ek(p,q)
    
    enumbers = []
    for pn in pnumbers:
        en = power_mod(int(pn),ek,n)
        enumbers.append(str(en))
    return enumbers,p,q,ek

def decrypt(enumbers,p,q,ek):
    print(enumbers)
    n = p*q
    phi = (p-1)*(q-1)
    dk = ek.inverse_mod(phi)

    pnumbers = []
    for en in enumbers:
        pn = power_mod(int(en),dk,n)
        pnumbers.append(str(pn))
    print(pnumbers)
    return pnumbers

In [10]:
encrypted_texts,p,q,ek = encrypt(encode('mathematicsisthequeenofscienceandnumbertheoryisthequeenofmathematicskfgauss'))
print(encrypted_texts)

['2311301815', '2311301913', '2919293018', '1527311515', '2425162913', '1915241315', '1124142431', '2312152830', '1815252835', '1929301815', '2731151524', '2516231130', '1815231130', '1913292116', '1711312929']
['136574537214', '16212428747', '160220065310', '99910379648', '13332058651', '143487673454', '83764700431', '22174833015', '207914439', '167944320404', '10874099725', '54313327318', '34516008944', '217384064310', '171046416121']


In [11]:
# encrypted_texts = ['5272281348','21089283929','3117723025','26844144908','22890519533',
# '26945939925','27395704341','2253724391','1481682985','2163791130',
# '13583590307','5838404872','12165330281','28372578777','7536755222']
#dts = decode(decrypt(encrypted_texts,187963,163841,48611))

In [12]:
decrypted_text = decode(decrypt(encrypted_texts,p,q,ek))
print(decrypted_text)

['136574537214', '16212428747', '160220065310', '99910379648', '13332058651', '143487673454', '83764700431', '22174833015', '207914439', '167944320404', '10874099725', '54313327318', '34516008944', '217384064310', '171046416121']
['2311301815', '2311301913', '2919293018', '1527311515', '2425162913', '1915241315', '1124142431', '2312152830', '1815252835', '1929301815', '2731151524', '2516231130', '1815231130', '1913292116', '1711312929']
mathematicsisthequeenofscienceandnumbertheoryisthequeenofmathematicskfgauss


# Best factoring algorithms
https://en.wikipedia.org/wiki/Integer_factorization

1. Dixon's algorithm
2. Continued fraction factorization
3. Quadratic sieve
4. Rational sieve
5. General number field sieve
6. Shank's square forms factorization

In [13]:
def crack(enumbers,ek,n):
    def find_key(ek,n):
        F = factor(n)       #TODO customize factoring algorithm
        p,q = F[0][0], F[1][0]
        phi = (p-1)*(q-1)
        dk = ek.inverse_mod(phi)
        return dk

    dk = find_key(ek,n)
    pnumbers = []
    for en in enumbers:
        pn = power_mod(int(en),dk,n)
        pnumbers.append(str(pn))
    
    return pnumbers

In [14]:
# crack test
cracked_text = decode(crack(encrypted_texts,ek,p*q))
print(cracked_text)

mathematicsisthequeenofscienceandnumbertheoryisthequeenofmathematicskfgauss
